## Script to move raw MACS files based on extracted footprints files 
**author:** Ingmar Nitze, Tabea Rettelbach

**contact:** ingmar.nitze@awi.de

**version date:** 2021-09-19


### Imports 

In [ ]:
import geopandas as gpd
import shutil
import os
import glob
import pandas as pd
from IPython.display import clear_output
import sys
import numpy as np
import tqdm
import zipfile
from pathlib import Path

In [ ]:
pwd = Path(os.getcwd())

### Functions 

In [ ]:
def prepare_df_for_mipps(path_footprints, path_infiles):
    # Load filtered footprints files
    df = gpd.read_file(path_footprints)
    
    flist = glob.glob(path_infiles + '/*/*.macs')
    
    df_full = pd.DataFrame()
    df_full['full_path'] = flist
    df_full['basename'] = pd.DataFrame(df_full['full_path'].apply(lambda x: os.path.basename(x)))
    # return Inner join of lists - create filtered list of filepaths 
    return df.set_index('Basename').join(df_full.set_index('basename'))

### Settings 
* prefer full/absolute paths
* Create processing template automatically

#### Variable Settings 

In [ ]:
# Set project directory here, where you want to process your data
PROJECT_DIR = r'D:\pix4d_Processing\Perma-X_2021\Villages\Kivalina' # SET Project output

# Set raw data dir here for the speicific image acquisition project
path_infiles = r'N:\response\Restricted_Airborne\MACs\2021_Perma-X_Alaska\01_raw_data\20210703-005031_01_Kivalina_1000m'

#### Fixed Settings 

In [ ]:
#CODE_DIR = r'D:\pix4d_Processing\code\MACS_tools'
CODE_DIR = pwd
MIPPS_DIR = r'C:\Program Files\DLR MACS-Box\bin'
MIPPS_BIN = r'..\tools\MACS\mipps.exe'

In [ ]:
mipps_script_nir = r'mipps_scripts\33552_all_taps_2018-09-26_12-58-15_modelbased.mipps'
mipps_script_right = r'mipps_scripts\33576_all_taps_2018-09-26_13-13-43_modelbased.mipps'
mipps_script_left = r'mipps_scripts\33577_all_taps_2018-09-26_13-21-24_modelbased.mipps'

mipps_script_nir = pwd / mipps_script_nir
mipps_script_right = pwd / mipps_script_right
mipps_script_left = pwd / mipps_script_left


In [ ]:
# Path of filtered footprints file (.shp file)
path_footprints = os.path.join(PROJECT_DIR, '02_studysites','footprints.shp')
outdir = os.path.join(PROJECT_DIR, '01_rawdata','tif')

#### Prepare processing dir 
* check if exists

In [ ]:
zippath = os.path.join(CODE_DIR, 'processing_folder_structure_template.zip')
nav_script_path = os.path.join(CODE_DIR, 'pix4dnav.py')

In [ ]:
with zipfile.ZipFile(zippath, 'r') as zip_ref:
    zip_ref.extractall(PROJECT_DIR)
shutil.copy(nav_script_path, outdir)

### Manual Selection of footprints 

Now select footprints and export selection as ***footprints.shp*** to ***02_footprints*** in your working directory

#### Load filtered footprints file 

In [ ]:
df_final = prepare_df_for_mipps(path_footprints, path_infiles)

In [ ]:
print("Total number of images:", len(df_final))
print("NIR images:", (df_final['Looking'] == 'center').sum())
print("RGB right images:", (df_final['Looking'] == 'right').sum())
print("RGB left images:", (df_final['Looking'] == 'left').sum())

#### Run Process 

In [ ]:
os.chdir(MIPPS_DIR)

In [ ]:
max_roll = 3 # Select maximum roll angle to avoid image issues
chunksize = 20

In [ ]:
looking = 'center'
q = (np.abs(df_final['Roll[deg]']) < max_roll) & (df_final['Looking'] == looking)
df_nir = df_final[q]
print(len(df_nir))
for df in tqdm.tqdm_notebook(np.array_split(df_nir, len(df_nir) // chunksize)):
    outlist = ' '.join(df['full_path'].values[:])
    s = f'{MIPPS_BIN} -c={mipps_script_nir} -o={outdir} -j=4 {outlist}'
    os.system(s)

In [ ]:
looking = 'right'
q = (np.abs(df_final['Roll[deg]']) < max_roll) & (df_final['Looking'] == looking)
df_right = df_final[q]
for df in tqdm.tqdm_notebook(np.array_split(df_right, len(df_right) // chunksize)):
    outlist = ' '.join(df['full_path'].values[:])
    s = f'{MIPPS_BIN} -c={mipps_script_right} -o={outdir} -j=4 {outlist}'
    os.system(s)

In [ ]:
looking = 'left'
q = (np.abs(df_final['Roll[deg]']) < max_roll) & (df_final['Looking'] == looking)
df_left = df_final[q]
for df in tqdm.tqdm_notebook(np.array_split(df_left, len(df_left) // chunksize)):
    outlist = ' '.join(df['full_path'].values[:])
    s = f'{MIPPS_BIN} -c={mipps_script_left} -o={outdir} -j=4 {outlist}'
    os.system(s)

#### Write exif information into all images 

#### Nav

In [ ]:
navfile = glob.glob(path_infiles + '/*nav.txt')[0]

In [ ]:
shutil.copy(navfile, os.path.join(outdir, 'nav.txt'))

In [ ]:
os.chdir(outdir)
os.system('python pix4dnav.py')